In [583]:
import random, copy
import cv2
import numpy as np
import random

In [584]:
def objective_function(position):
    # Create binary mask based on threshold value
    threshold_value = position
    _, mask = cv2.threshold(img, threshold_value, 255, cv2.THRESH_BINARY)

    # Compute percentage of foreground pixels in mask
    n_pixels = mask.flatten().shape[0]
    n_foreground_pixels = np.count_nonzero(mask == 255)
    if n_pixels > 0:
        percent_foreground_pixels = n_foreground_pixels / n_pixels
    else:
        percent_foreground_pixels = 0

    # Return negative of percentage of foreground pixels as fitness value
    return percent_foreground_pixels

In [585]:
### POPULATION LEVEL ###
def gpso_search():
    population = create_pop()
    personal_best_population = copy.copy(population)
    global_best, global_best_fitness = get_best_pop(personal_best_population)
    #print(global_best_fitness)
    for _ in range(NUM_GENERATIONS):
       # print(population,'\n')
        #population = convex_combination_pop(population, personal_best_population, [global_best]*POPULATION_SIZE)
        population = mutation_pop(population)
        personal_best_population = selection_pop(personal_best_population, population)
        global_best, global_best_fitness = get_best_pop(personal_best_population)
        #print(global_best_fitness)
    return (global_best, global_best_fitness) 

In [586]:
### INDIVIDUAL LEVEL ###

def create_pop():
    return [ create_ind() for _ in range(POPULATION_SIZE) ]
def get_best_pop(population):
    a=[]
    population=create_pop()
    for pop in population:
        a.append([objective_function(pops) for pops in pop])
    best_ind = max(a, key=evaluate_ind)
    for i in range(POPULATION_SIZE):
        if(best_ind==a[i]):
            return population[i],evaluate_ind(best_ind)

# def get_best_pop(population):
#     best_ind = max(population, key=evaluate_ind)
#    # print(best_ind)
# #     a=[objective_function(best_inds) for best_inds in best_ind]
#     a=[image_entropy(best_inds) for best_inds in best_ind]
#     return best_ind,evaluate_ind(a)

def convex_combination_pop(pop1, pop2, pop3):
    return [ convex_combination_ind([ind1, ind2, ind3]) for (ind1, ind2, ind3) in zip(pop1, pop2, pop3) ]

def mutation_pop(population):
    return [ mutation_ind(individual) for individual in population ]

def selection_pop(pop1, pop2):
    return [ max(ind1, ind2, key=evaluate_ind) for (ind1, ind2) in zip(pop1, pop2) ]



In [587]:
### REPRESENTATION LEVEL ###

def create_ind():
    return [ random.randint(0, 255) for _ in range(INDIVIDUAL_SIZE) ]

def evaluate_ind(individual): #one_max
    return sum(individual)

def convex_combination_ind(mating_pool):
    transposed_mating_pool=zip(*mating_pool)
    return map(random.choice, transposed_mating_pool)

def mutation_ind(individual):
    return [ 1-bit if random.random()<1.0/INDIVIDUAL_SIZE else bit for bit in individual ]


In [588]:
import math

def image_entropy(position):
    _, mask = cv2.threshold(img, position, 255, cv2.THRESH_BINARY)
    segmented_image = cv2.bitwise_and(img, img, mask=mask)
    # Calculate the histogram
    hist = cv2.calcHist([segmented_image], [0], None, [256], [0, 256])

    # Normalize the histogram so it sums to 1
    hist /= float(segmented_image.size)

    # Calculate the entropy
    entropy = 0
    for item in hist:
        if item != 0:
            entropy -= item * math.log2(item)

    return entropy

In [589]:
best_ind = min(create_pop(), key=evaluate_ind)

In [590]:
[objective_function(best_inds) for best_inds in best_ind]

[0.848191774662363, 0.9789709495591848, 0.9927333750863163, 0.8277809895456955]

In [591]:
best_ind

[114, 15, 3, 118]

In [592]:
selection_pop(create_pop(),create_pop())

[[190, 82, 231, 92],
 [173, 114, 30, 170],
 [185, 110, 192, 124],
 [191, 201, 236, 88],
 [242, 239, 138, 62],
 [21, 5, 161, 142],
 [67, 87, 192, 244],
 [163, 214, 95, 0],
 [178, 254, 155, 56],
 [143, 154, 73, 241],
 [148, 220, 194, 23],
 [112, 195, 108, 245],
 [232, 118, 2, 144],
 [100, 11, 208, 137],
 [173, 129, 151, 187],
 [237, 114, 47, 86],
 [54, 242, 93, 180],
 [203, 112, 86, 168],
 [47, 65, 139, 176],
 [68, 177, 51, 202],
 [97, 132, 245, 235],
 [89, 255, 170, 199],
 [205, 235, 87, 107],
 [201, 250, 182, 253],
 [206, 147, 239, 11]]

In [593]:
min(create_pop(), key=evaluate_ind)

[97, 89, 91, 20]

In [594]:
get_best_pop(create_pop())

([9, 6, 44, 52], 3.8721796662973134)

In [595]:
a=[]
population=create_pop()
for pop in population:
    a.append([objective_function(pops) for pops in pop])
best_ind = max(a, key=evaluate_ind)
for i in range(POPULATION_SIZE):
    if(best_ind==a[i]):
        print(population[i])    
print(best_ind,evaluate_ind(best_ind))

[3, 16, 2, 40]
[0.9927333750863163, 0.977826757238522, 0.9937410672704791, 0.954577572224631] 3.9188787718199487


In [596]:
for i in range(POPULATION_SIZE):
    if(best_ind==a[i]):
        print(population[i])

[3, 16, 2, 40]


In [597]:
img = cv2.imread('1.jpg',0)

### EXPERIMENTS ###

NUM_GENERATIONS = 100   
POPULATION_SIZE = 25
INDIVIDUAL_SIZE = 4
global_best2, global_best_fitness2=gpso_search()

In [598]:
print(global_best2)

[106, 59, 11, 62]


In [599]:
for i in range(INDIVIDUAL_SIZE):
    threshold_value = int(global_best2[i])
    _, mask = cv2.threshold(img, threshold_value, 255, cv2.THRESH_BINARY)
    segmented_image = cv2.bitwise_and(img, img, mask=mask)
    cv2.imshow("Segmented Image"+str(i), segmented_image)
    cv2.imshow("mask"+str(i),mask)
cv2.waitKey(0)
cv2.destroyAllWindows()